In [1]:
import os

In [2]:
%pwd

'd:\\TextSummarizationProject\\End-to-end-Text-Summarizer-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\TextSummarizationProject\\End-to-end-Text-Summarizer-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataReceiverConfig: #Bu bir python actual classı değildir bir Data Class'tır.
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

#Bu bir fonksiyonun return type'ı gibidir. Mesela bir "DataIngestionConfig" instance oluşturduğumuz zaman bu variableları return edecek. Bu variableları
#gerektiği zamanlarda kullanacağız.
#Mesela "root_dir" çağırırsak "config.yaml" dosyası içerisindeki "artifacts/data_ingestion" pathini return edecek.
#Bu bir ENTİTY'DİR.

In [11]:
#Şimdi "README.md" dosyasında yer alan 4. adım olan "4. Update the configuration manager in src config" adımda sıra. 
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [21]:
class ConfigurationManager: 
    def __init__( self, config_filepath = FILE_PATH_CONFIG, params_filepath = FILE_PATH_PARAMS):
        
        #common.py dosyasındaki read_yaml() fonksiyonu ile yaml dosyasını okuyoruz.
        self.config = read_yaml(config_filepath) #Aslında Path("config/config.yaml")
        self.params = read_yaml(params_filepath) #Aslında Path("params.yaml")

        create_directories([self.config.artifacts_root]) #artifacts_root: artifacts. Bu method ile "artifacts" isimli folder otomatik olarak oluşturulur.
        #"." kullanarak çağırmayı "ConfigBox"a borçluyuz.

    def get_config_data_receiver(self):
        config = self.config.data_receiver #Alttakine erişiyoruz. read_yaml(config_filepath) ile bu config.yaml dosyasını okuduk.
        #data_data_receiver:
            #root_dir: artifacts/data_ingestion
            #source_URL: https://github.com/kdenizakin/End-to-end-Text-Summarizer-Project/raw/main/text_summarizer_dataset.zip #bu URL'den datayı indirecek.
            #local_data_file: artifacts/data_ingestion/data.zip #datayı bu foldera ve "data.zip" ismiyle indirecek.
            #unzip_dir: artifacts/data_ingestion #datanın unzip edilmiş hali buraya download edilecek.

        create_directories([config.root_dir])

        data_receiver_config = DataReceiverConfig(
            root_dir = config.root_dir, # config.root_dir = "artifacts/data_receiver". Yani artifacts/data_receiver isimli bir dosya oluşturulacak.
            source_URL = config.source_URL,
            local_data_file = Path(config.local_data_file),
            unzip_dir = Path(config.unzip_dir)
        )

        return data_receiver_config

In [16]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [17]:
class DataReceiver:
    def __init__(self, data_receiver_config: DataReceiverConfig):
        self.config = data_receiver_config

    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(self.config.source_URL,self.config.local_data_file) #config.yaml'dan tüm pathleri alıyoruz.
            logger.info(f"Downloaded data from {self.config.source_URL} to {self.config.local_data_file} with information: {headers}")
        else:
            logger.info(f"Data already exists at {self.config.local_data_file}")

    #Datayı download ettikten sonra extract edilmesi de gerekmektedir.
    def extract_data(self):
        folder_path_to_unzip = self.config.unzip_dir
        os.makedirs(folder_path_to_unzip, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(folder_path_to_unzip)



In [23]:
#Burası pipeline
try:
    config_manager_obj = ConfigurationManager()
    data_receiver_config =  config_manager_obj.get_config_data_receiver()
    data_receiver = DataReceiver(data_receiver_config=data_receiver_config)
    data_receiver.download_data()
    data_receiver.extract_data()
    logger.info(f"Extracted data from {data_receiver_config.local_data_file} to {data_receiver_config.unzip_dir} with size {get_size(data_receiver_config.unzip_dir)}")
except Exception as e:
    raise e

[2024-05-20 07:47:56,161: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-20 07:47:56,163: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-20 07:47:56,166: INFO: common: Created directory at artifacts]
[2024-05-20 07:47:56,168: INFO: common: Created directory at artifacts/data_receiver]
[2024-05-20 07:47:58,223: INFO: 1475340574: Downloaded data from https://github.com/kdenizakin/End-to-end-Text-Summarizer-Project/raw/main/text_summarizer_dataset.zip to artifacts\data_receiver\data.zip with information: Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8AE0:1E7558:3C19220:3F6651E:664AD5FA
Accept